# Import bibliotek oraz wczytanie danych

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
import random
import scipy.sparse as sp
from libreco.data import random_split, DatasetPure, split_by_ratio_chrono
from libreco.algorithms import LightGCN
from libreco.evaluation import evaluate

In [523]:
df = pd.read_csv(#"C:\\Users\\marci\\OneDrive\\Pulpit\\archive\\interactions.csv",
                 "C:\\Users\\fpazi\\Desktop\\projekty\\olx\\interactions.csv",
                 sep=',')

## Wstępna obróbka danych

Podział wartości z kolumny `event` tak aby móc dostosować naszą ramkę danych do zadania. 

In [524]:
rating = {
    'click': 1,
    'bookmark': 2,
    'chat_click': 3,
    'contact_phone_click_1': 3,
    'contact_partner_click': 3,
    'contact_phone_click_2': 3,
    'contact_phone_click_3': 3,
    'contact_chat': 3
}
df['event'] = df['event'].map(rating)
df = df.rename(columns={'event': 'label'})
df = df.rename(columns={'timestamp': 'time'})

Ustawiamy liczbę interakcji użytkowników od największej do najmniejszej i sprawdzamy jak wyglądają te liczby.

In [525]:
user_interaction_counts = df['user'].value_counts()

# Sortowanie użytkowników według liczby interakcji od największej do najmniejszej
sorted_users = user_interaction_counts.sort_values(ascending=False)

# Wyświetlenie posortowanych użytkowników
print(sorted_users.head())

user
750207     1310
2358298    1308
3057718    1308
88606      1307
2016978    1303
Name: count, dtype: int64


Pozbywamy się użytkowników z liczbą interakcji mniejszą niż 100. Mamy nadzieje odsiać w ten sposób użytkowników, którzy nie są regularnymi użytkownikami.

In [526]:
reg_users = user_interaction_counts[user_interaction_counts >= 100]

Pozbywamy się outlierów korzystając z metody 8 sigm.

In [527]:
users_to_keep = reg_users[reg_users <= reg_users.mean() + 8*reg_users.std()].index
df = df[df['user'].isin(users_to_keep)]

In [528]:
items_interaction_counts = df['item'].value_counts()

# Sortowanie użytkowników według liczby interakcji od największej do najmniejszej
sorted_items = items_interaction_counts.sort_values(ascending=False)

# Wyświetlenie posortowanych użytkowników
print(sorted_items.head())

item
44153     7276
52054     7080
179685    6961
66717     6957
16819     6909
Name: count, dtype: int64


Pozbywamy się przedmiotów o mniejszej liczbie interakcji niż 1000. Mamy nadzieje, by w ten sposób, zachować przedmioty, które cieszą się przynajmniej umiarkowanym sukcesem.

In [529]:
popular_items = items_interaction_counts[items_interaction_counts >= 1000]

Pozbywamy się outlierów korzystając z metody 9 sigm.

In [530]:
items_to_keep = popular_items[popular_items <= popular_items.mean() + 9*popular_items.std()].index
df = df[df['item'].isin(items_to_keep)]

Dla tej kombinacji sigm zaobserwowałem najwyższe gęstość danych.

Pierwsza filtracja ramki danych. Wybieramy 20% przedmiotów o największej liczbie interakcji.

In [531]:
# Obliczenie liczby interakcji dla każdego przedmiotu
item_interaction_counts = df['item'].value_counts()

# Obliczenie kwantyla dla top 20% przedmiotów
top_20_threshold = item_interaction_counts.quantile(0.8)

# Wybór przedmiotów, które są w top 20% o największej liczbie interakcji
top_20_items = item_interaction_counts[item_interaction_counts >= top_20_threshold].index

# Przefiltrowanie oryginalnej ramki danych, aby pozostawić tylko wybranych użytkowników
filtered_df = df[df['item'].isin(top_20_items)]

In [532]:
item_interaction_counts = filtered_df['item'].value_counts()

# Sortowanie użytkowników według liczby interakcji od największej do najmniejszej
sorted_items = item_interaction_counts.sort_values(ascending=False)

# Wyświetlenie posortowanych użytkowników
print(sorted_items.head())

item
44153     7276
52054     7080
179685    6961
66717     6957
16819     6909
Name: count, dtype: int64


To samo robimy dla 20% użytkowników o największej liczbie interakcji.

In [533]:
user_interaction_counts = filtered_df['user'].value_counts()

# Sortowanie użytkowników według liczby interakcji od największej do najmniejszej
sorted_users = user_interaction_counts.sort_values(ascending=False)

# Wyświetlenie posortowanych użytkowników
print(sorted_users.head())

user
1288580    1054
1948352     537
2090363     484
3049290     468
239584      396
Name: count, dtype: int64


In [534]:
# Obliczenie liczby interakcji dla każdego użytkownika
user_interaction_counts = df['user'].value_counts()

# Obliczenie kwantyla dla top 20% użytkowników
top_20_threshold = user_interaction_counts.quantile(0.8)

# Wybór użytkowników, którzy są w top 20% o największej liczbie interakcji
top_20_users = user_interaction_counts[user_interaction_counts >= top_20_threshold].index

# Przefiltrowanie oryginalnej ramki danych, aby pozostawić tylko wybranych użytkowników
filtered_df = filtered_df[filtered_df['user'].isin(top_20_users)]

In [535]:
item_interaction_counts = filtered_df['item'].value_counts()

# Sortowanie użytkowników według liczby interakcji od największej do najmniejszej
sorted_items = item_interaction_counts.sort_values(ascending=False)

# Wyświetlenie posortowanych użytkowników
print(sorted_items.head())

user_interaction_counts = filtered_df['user'].value_counts()

# Sortowanie użytkowników według liczby interakcji od największej do najmniejszej
sorted_userss = user_interaction_counts.sort_values(ascending=False)

# Wyświetlenie posortowanych użytkowników
print(sorted_users.head())

item
66717     5250
44153     5098
16819     5049
171495    4957
179685    4912
Name: count, dtype: int64
user
1288580    1054
1948352     537
2090363     484
3049290     468
239584      396
Name: count, dtype: int64


Porównujemy proporcję przedmioty / użytkownicy w wejściowej ramce danych oraz przefiltrowanej. Jak widzimy, proporcja ta jest prawie identyczna.

In [536]:
count_users_before = df['user'].nunique()
count_items_before = df['item'].nunique()
count_users = filtered_df['user'].nunique()
print(count_users)
count_items = filtered_df['item'].nunique()
print(count_items)
print(count_items/count_users * 100)
count_items_before/count_users_before * 100

25060
659
2.6296887470071826


2.6695544253279144

Kolejna filtracja ramki danych. Teraz losowo wybieramy 20% unikatowych przedmiotów i użytkowników tak, aby maszyny, na których pracujemy nie miały problemów z pamięcią.

In [537]:
# Procent użytkowników i przedmiotów do wyboru
sample_percent = 0.20

unique_users_count = filtered_df['user'].nunique()
unique_items_count = filtered_df['item'].nunique()

# Wybór x% unikatowych użytkowników
sampled_users = filtered_df.drop_duplicates('user').sample(frac=sample_percent, random_state=2024)

# Wybór x% unikatowych przedmiotów
sampled_items = filtered_df.drop_duplicates('item').sample(frac=sample_percent, random_state=2024)

# Przefiltrowanie przefiltrowanej ramki danych, aby pozostawić tylko wybrane użytkowniki
filtered_df_users = filtered_df[filtered_df['user'].isin(sampled_users['user'])]

# Przefiltrowanie wynikowej ramki danych, aby pozostawić tylko wybrane przedmioty
filtered_df = filtered_df_users[filtered_df_users['item'].isin(sampled_items['item'])]

Znów sprawdzmy proporcję przedmioty / użytkownicy. 

In [538]:
count_users = filtered_df['user'].nunique()
print(count_users)
count_items = filtered_df['item'].nunique()
print(count_items)
print(count_items/count_users * 100)
count_items_before/count_users_before * 100

4607
132
2.8652051226394617


2.6695544253279144

In [539]:
user_interaction_counts = filtered_df['user'].value_counts()

# Sortowanie użytkowników według liczby interakcji od największej do najmniejszej
sorted_users = user_interaction_counts.sort_values(ascending=False)

# Wyświetlenie posortowanych użytkowników
print(sorted_users.head())

user
3243167    95
805994     90
2470737    88
3109422    80
3006708    79
Name: count, dtype: int64


In [540]:
# # user_interaction_counts = df['user'].value_counts()
# # item_interaction_counts = df['item'].value_counts()

# # # Wybór 20% użytkowników o największej liczbie interakcji
# # top_users = user_interaction_counts.head(int(len(user_interaction_counts) * 0.2))

# # # Wybór 20% przedmiotów o największej liczbie interakcji
# # top_items = item_interaction_counts.head(int(len(item_interaction_counts) * 0.2))

# # Wybór x% użytkowników i przedmiotów do próbkowania
# sample_percent = 0.2  # Możesz zmieniać ten procent w zakresie od 0.01 do 0.5

# # Konwertowanie indeksu na ramkę danych dla użytkowników i przedmiotów
# top_users_df = top_users.reset_index()
# top_users_df.columns = ['user', 'interactions']
# top_items_df = top_items.reset_index()
# top_items_df.columns = ['item', 'interactions']

# # Wybór x% użytkowników
# sampled_users = filtered_df.sample(frac=sample_percent, random_state=42)

# # Wybór x% przedmiotów
# sampled_items = filtered_df.sample(frac=sample_percent, random_state=42)

# # Wyświetlenie wybranych użytkowników i przedmiotów
# print("Wybrane użytkownicy:")
# print(sampled_users.head())
# print("\nWybrane przedmioty:")
# print(sampled_items.head())

In [541]:
# filtered_df = df[df['user'].isin(sampled_users['user']) & df['item'].isin(sampled_items['item'])]

# # Wyświetlenie przefiltrowanej ramki danych
# print(filtered_df.head())

In [542]:
# user_interaction_counts = df.groupby('user')['label'].count()
# item_interaction_counts = df.groupby('item')['label'].count()
# user_mean = user_interaction_counts.mean()
# user_std = user_interaction_counts.std()
# item_mean = item_interaction_counts.mean()
# item_std = item_interaction_counts.std()

# # Ustalenie granicy dla wartości odstających (trzy odchylenia standardowe od średniej)
# user_outlier_threshold = user_mean + 3 * user_std
# item_outlier_threshold = item_mean + 3 * item_std

# # Odrzucenie outlierów dla użytkowników
# users_no_outliers = df.groupby('user').filter(lambda x: x['label'].count() < user_outlier_threshold)

# # Odrzucenie outlierów dla przedmiotów
# items_no_outliers = df.groupby('item').filter(lambda x: x['label'].count() < item_outlier_threshold)

# # Ograniczenie ramki danych do użytkowników i przedmiotów bez outlierów
# df_no_outliers = df[df['user'].isin(users_no_outliers['user'].unique()) & df['item'].isin(items_no_outliers['item'].unique())]

In [543]:
# count_users_before = df['user'].nunique()
# count_items_before = df['item'].nunique()
# count_users = df_no_outliers['user'].nunique()
# print(count_users)
# count_items = df_no_outliers['item'].nunique()
# print(count_items)
# print(count_items/count_users * 100)
# count_items_before/count_users_before * 100

In [544]:
# user_interaction_counts = df_no_outliers.groupby('user')['label'].count()
# item_interaction_counts = df_no_outliers.groupby('item')['label'].count()

# # Obliczenie liczby unikalnych wartości, które chcemy zachować
# n_unique_values_to_keep = int(len(item_interaction_counts) * 0.2)

# # Wybieranie 20% najczęściej występujących unikalnych wartości 'item'
# most_common_values = item_interaction_counts.head(n_unique_values_to_keep).index.tolist()

# # Ograniczenie ramki danych do wierszy zawierających te wartości
# filtered_df = df_no_outliers[df_no_outliers['item'].isin(most_common_values)]

# n_unique_values_to_keep_user = int(len(user_interaction_counts) * 0.2)

# # Wybieranie 20% najczęściej występujących unikalnych wartości 'item'
# most_common_values = user_interaction_counts.head(n_unique_values_to_keep_user).index.tolist()

# # Ograniczenie ramki danych do wierszy zawierających te wartości
# filtered_df = filtered_df[filtered_df['user'].isin(most_common_values)]

In [545]:
# count_users = filtered_df['user'].nunique()
# print(count_users)
# count_items = filtered_df['item'].nunique()
# print(count_items)
# print(count_items/count_users * 100)
# count_items_before/count_users_before * 100

In [546]:
# grouped = filtered_df.groupby(['user', 'item'])['label'].nunique()

# # Znajdowanie indeksów wierszy, które spełniają warunek (tylko jedna unikalna wartość 'event')
# rows_to_drop = grouped[grouped == 1].index

# # Usuwanie wierszy, które spełniają warunek
# filtered_df = filtered_df[~filtered_df.set_index(['user', 'item']).index.isin(rows_to_drop)]

## Podział przefiltrowanej ramki danych na zbiór treningowy i testowy

In [547]:
distinct_users = filtered_df['user'].unique()

num_users_group1 = int(0.7 * distinct_users.shape[0])  # 7%
num_users_group2 = int(0.3 * distinct_users.shape[0])  # 3%

group1_users = random.sample(list(distinct_users), num_users_group1)
remaining_users = list(set(distinct_users) - set(group1_users))
group2_users = random.sample(remaining_users, num_users_group2)
#group3_users = list(set(remaining_users) - set(group2_users))

group1_df = filtered_df[filtered_df['user'].isin(group1_users)]
group2_df = filtered_df[filtered_df['user'].isin(group2_users)]
#group3_df = filtered_df[filtered_df['user'].isin(group3_users)]

In [548]:
# Obliczenie liczby interakcji dla każdego przedmiotu
item_interaction_counts = group1_df['item'].value_counts()

# Obliczenie kwantyla dla top 20% przedmiotów
top_20_threshold = item_interaction_counts.quantile(0.8)

# Wybór przedmiotów, które są w top 20% o największej liczbie interakcji
top_20_items = item_interaction_counts[item_interaction_counts >= top_20_threshold].index

# Przefiltrowanie oryginalnej ramki danych, aby pozostawić tylko wybranych użytkowników
group1_df = group1_df[group1_df['item'].isin(top_20_items)]

In [549]:
# unique_users_count = group1_df['user'].nunique()
# unique_items_count = group1_df['item'].nunique()

# Wybór x% unikatowych użytkowników
#sampled_users = filtered_df.drop_duplicates('user').sample(frac=sample_percent, random_state=2024)

# # Wybór x% unikatowych przedmiotów
# sampled_items = group1_df.drop_duplicates('item').sample(frac=0.2, random_state=2024)

# Przefiltrowanie przefiltrowanej ramki danych, aby pozostawić tylko wybrane użytkowniki
#filtered_df_users = filtered_df[filtered_df['user'].isin(sampled_users['user'])]

# # Przefiltrowanie wynikowej ramki danych, aby pozostawić tylko wybrane przedmioty
# group1_df = group1_df[group1_df['item'].isin(sampled_items['item'])]

# user_interaction_counts = group1_df.groupby('user')['label'].count()
# item_interaction_counts = group1_df.groupby('item')['label'].count()

# # Obliczenie liczby unikalnych wartości, które chcemy zachować
# n_unique_values_to_keep = int(len(item_interaction_counts) * 0.2)

# # Wybieranie 20% najczęściej występujących unikalnych wartości 'item'
# most_common_values = item_interaction_counts.head(n_unique_values_to_keep).index.tolist()

# # Ograniczenie ramki danych do wierszy zawierających te wartości
# filtered_g1 = group1_df[group1_df['item'].isin(most_common_values)]

# n_unique_values_to_keep_user = int(len(user_interaction_counts) * 0.8)

# # Wybieranie 20% najczęściej występujących unikalnych wartości 'item'
# most_common_values = user_interaction_counts.head(n_unique_values_to_keep_user).index.tolist()

# # Ograniczenie ramki danych do wierszy zawierających te wartości
# filtered_g1 = filtered_g1[filtered_g1['user'].isin(most_common_values)]

In [550]:
count_users = group1_df['user'].nunique()
print(count_users)
count_items = group1_df['item'].nunique()
print(count_items)
print(count_items/count_users * 100)
count_items_before/count_users_before * 100

1898
27
1.422550052687039


2.6695544253279144

In [551]:
user_interaction_counts = group1_df['user'].value_counts()

# Sortowanie użytkowników według liczby interakcji od największej do najmniejszej
sorted_users = user_interaction_counts.sort_values(ascending=False)

# Wyświetlenie posortowanych użytkowników
print(sorted_users.head())

user
805994     89
2470737    67
595339     57
2527416    55
1350236    54
Name: count, dtype: int64


In [552]:
# def split_data_by_user_quantile(df, quant):
# #     # Group the data by user
#      grouped = df.groupby('user')

# #     # Initialize empty lists to store data below and above quantile threshold for each user
#      below_quantile_list = []
#      above_quantile_list = []

# #     # Iterate over each user group
#      for user, group_data in grouped:
#          # Calculate the 70th quantile of the timestamp column for the current user
#          quantile_user = group_data['time'].quantile(quant)

# #         # Split the data for the current user into two sets based on the quantile threshold
#          below_quantile = group_data[group_data['time'] <= quantile_user]
#          above_quantile = group_data[group_data['time'] > quantile_user]

# #         # Append the sets to the respective lists
#          below_quantile_list.append(below_quantile)
#          above_quantile_list.append(above_quantile)

# #     # Concatenate the sets for all users into DataFrames
#      below_quantile_df = pd.concat(below_quantile_list)
#      above_quantile_df = pd.concat(above_quantile_list)

#      return below_quantile_df, above_quantile_df

# # # Split data for each group based on user-specific quantile threshold
# group1_below_user_quantile, group1_above_user_quantile = split_data_by_user_quantile(filtered_g1, 0.7)
# # group2_below_user_quantile, group2_above_user_quantile = split_data_by_user_quantile(group2_df, 0.7)

# # # Display the shapes of the resulting sets
# print("Group 1 - Below or equal  70th user-specific quantile:", group1_below_user_quantile.shape)
# print("Group 1 - Above to 70th user-specific quantile:", group1_above_user_quantile.shape)
# #print("Group 2 - Below or equal 70th user-specific quantile:", group2_below_user_quantile.shape)
# #print("Group 2 - Above to 70th user-specific quantile:", group2_above_user_quantile.shape)

In [553]:
# print(group1_below_user_quantile['item'].nunique())
# group1_above_user_quantile['item'].nunique()

In [554]:
# unique_items_b = set(group1_below_user_quantile['item'].unique())
# unique_items_a = set(group1_above_user_quantile['item'].unique())

# unique_items_only_in_a = unique_items_a.difference(unique_items_b)
# unique_items_only_in_b = unique_items_b.difference(unique_items_a)

# print(len(unique_items_only_in_a))
# print(len(unique_items_only_in_b))

Implementujemy kod z biblioteki LibRecommender dzielący dane na zbiór treningowy i ewaluacyjny.

In [555]:
# train_data = group1_below_user_quantile
# eval_data = group1_above_user_quantile
train_data, eval_data = split_by_ratio_chrono(filtered_df,test_size=0.2)
train_data, eval_data = split_by_ratio_chrono(group1_df,test_size=0.2)
train_data, data_info = DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
#test_data = DatasetPure.build_testset(test_data)
print(data_info)

n_users: 1898, n_items: 27, data density: 18.3331 %


In [563]:
lightgcn = LightGCN(
    task="ranking",
    data_info=data_info,
    loss_type="bpr",
    embed_size=32,
    n_epochs=3,
    lr=1e-2,
    batch_size=2048,
    num_neg=1,
    device="cuda"
)

In [564]:
%%time
# można też spróbować %%timeit jeśli kod się będzie w miarę krótko wykonywał (liczy średnii czas wykonywania komórki) 
lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "precision", "recall", "ndcg", "map"]
)

Training start time: 2024-03-03 23:03:50


train: 100%|██████████| 5/5 [00:00<00:00, 90.99it/s]


Epoch 1 elapsed: 0.057s
	 train_loss: 0.6728


eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 997.46it/s]
c:\Users\fpazi\anaconda3\envs\olx\Lib\site-packages\sklearn\metrics\_classification.py:2910: FutureWarning: Setting the eps parameter is deprecated and will be removed in 1.5. Instead eps will always havea default value of `np.finfo(y_pred.dtype).eps`.
  warnings.warn(
eval_listwise: 100%|██████████| 4/4 [00:00<00:00, 80.01it/s]


	 eval log_loss: 0.6609
	 eval precision@10: 0.0290
	 eval recall@10: 0.2077
	 eval ndcg@10: 0.1516
	 eval map@10: 0.1162


...

	 eval log_loss: 0.5892
	 eval precision@10: 0.0289
	 eval recall@10: 0.2085
	 eval ndcg@10: 0.1556
	 eval map@10: 0.1214


train: 100%|██████████| 5/5 [00:00<00:00, 108.27it/s]


Epoch 3 elapsed: 0.049s
	 train_loss: 0.4219


eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 499.92it/s]
c:\Users\fpazi\anaconda3\envs\olx\Lib\site-packages\sklearn\metrics\_classification.py:2910: FutureWarning: Setting the eps parameter is deprecated and will be removed in 1.5. Instead eps will always havea default value of `np.finfo(y_pred.dtype).eps`.
  warnings.warn(
eval_listwise: 100%|██████████| 4/4 [00:00<00:00, 73.10it/s]


	 eval log_loss: 0.5162
	 eval precision@10: 0.0289
	 eval recall@10: 0.2090
	 eval ndcg@10: 0.1569
	 eval map@10: 0.1230
CPU times: total: 219 ms
Wall time: 557 ms


In [565]:
%%time
lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "precision", "recall", "ndcg", "map"],
    k=5
)

Training start time: 2024-03-03 23:03:50


train: 100%|██████████| 5/5 [00:00<00:00, 93.10it/s]


Epoch 1 elapsed: 0.055s
	 train_loss: 0.2883


eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 998.88it/s]
c:\Users\fpazi\anaconda3\envs\olx\Lib\site-packages\sklearn\metrics\_classification.py:2910: FutureWarning: Setting the eps parameter is deprecated and will be removed in 1.5. Instead eps will always havea default value of `np.finfo(y_pred.dtype).eps`.
  warnings.warn(
eval_listwise: 100%|██████████| 4/4 [00:00<00:00, 81.56it/s]


	 eval log_loss: 0.5001
	 eval precision@5: 0.0397
	 eval recall@5: 0.1404
	 eval ndcg@5: 0.1321
	 eval map@5: 0.1133


...

	 eval log_loss: 0.5380
	 eval precision@5: 0.0405
	 eval recall@5: 0.1438
	 eval ndcg@5: 0.1348
	 eval map@5: 0.1154


train: 100%|██████████| 5/5 [00:00<00:00, 116.23it/s]


Epoch 3 elapsed: 0.045s
	 train_loss: 0.1924


eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 663.97it/s]
c:\Users\fpazi\anaconda3\envs\olx\Lib\site-packages\sklearn\metrics\_classification.py:2910: FutureWarning: Setting the eps parameter is deprecated and will be removed in 1.5. Instead eps will always havea default value of `np.finfo(y_pred.dtype).eps`.
  warnings.warn(
eval_listwise: 100%|██████████| 4/4 [00:00<00:00, 71.93it/s]


	 eval log_loss: 0.5868
	 eval precision@5: 0.0420
	 eval recall@5: 0.1509
	 eval ndcg@5: 0.1428
	 eval map@5: 0.1235
CPU times: total: 156 ms
Wall time: 447 ms


In [566]:
%%time
lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "precision", "recall", "ndcg", "map"],
    k=2
)

Training start time: 2024-03-03 23:03:51


train: 100%|██████████| 5/5 [00:00<00:00, 113.26it/s]


Epoch 1 elapsed: 0.046s
	 train_loss: 0.1791


eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 816.97it/s]
c:\Users\fpazi\anaconda3\envs\olx\Lib\site-packages\sklearn\metrics\_classification.py:2910: FutureWarning: Setting the eps parameter is deprecated and will be removed in 1.5. Instead eps will always havea default value of `np.finfo(y_pred.dtype).eps`.
  warnings.warn(
eval_listwise: 100%|██████████| 4/4 [00:00<00:00, 80.72it/s]


	 eval log_loss: 0.6199
	 eval precision@2: 0.0707
	 eval recall@2: 0.1034
	 eval ndcg@2: 0.1178
	 eval map@2: 0.1102


...

	 eval log_loss: 0.6369
	 eval precision@2: 0.0744
	 eval recall@2: 0.1080
	 eval ndcg@2: 0.1237
	 eval map@2: 0.1155


train: 100%|██████████| 5/5 [00:00<00:00, 117.66it/s]


Epoch 3 elapsed: 0.045s
	 train_loss: 0.1474


eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 724.66it/s]
c:\Users\fpazi\anaconda3\envs\olx\Lib\site-packages\sklearn\metrics\_classification.py:2910: FutureWarning: Setting the eps parameter is deprecated and will be removed in 1.5. Instead eps will always havea default value of `np.finfo(y_pred.dtype).eps`.
  warnings.warn(
eval_listwise: 100%|██████████| 4/4 [00:00<00:00, 86.76it/s]

	 eval log_loss: 0.6440
	 eval precision@2: 0.0733
	 eval recall@2: 0.1056
	 eval ndcg@2: 0.1226
	 eval map@2: 0.1145
CPU times: total: 234 ms
Wall time: 525 ms


Wszystko poniżej tego kawałka kodu to zabawy w zmianę funkcji straty, zmiana parametrów itd itp.

In [567]:
HAMUJ SIĘ

SyntaxError: invalid syntax (233987486.py, line 1)

In [ ]:
# evaluate(
#     model=lightgcn,
#     data=group2_df,
#     neg_sampling=True,
#     metrics=["loss", "precision", "recall", "ndcg", "map"],
#     k=10
# )

In [ ]:
lightgcn = LightGCN(
    task="ranking",
    data_info=data_info,
    loss_type="cross_entropy",
    embed_size=32,
    n_epochs=5,
    lr=0.001,
    batch_size=512,
    num_neg=1,
    device="cuda"
)

lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "precision", "recall", "ndcg", "map"]
)

In [ ]:
lightgcn = LightGCN(
    task="ranking",
    data_info=data_info,
    loss_type="cross_entropy",
    embed_size=64,
    n_epochs=5,
    lr=1e-2,
    batch_size=2048,
    num_neg=1,
    device="cuda"
)

lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "precision", "recall", "ndcg", "map"]
)

In [ ]:
lightgcn = LightGCN(
    task="ranking",
    data_info=data_info,
    loss_type="focal",
    embed_size=64,
    n_epochs=5,
    lr=1e-2,
    batch_size=2048,
    num_neg=1,
    device="cuda"
)

lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "precision", "recall", "ndcg", "map"]
)

In [ ]:
lightgcn = LightGCN(
    task="ranking",
    data_info=data_info,
    loss_type="max_margin",
    embed_size=64,
    n_epochs=5,
    lr=1e-2,
    batch_size=2048,
    num_neg=1,
    device="cuda"
)

lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "precision", "recall", "ndcg", "map"]
)